# Requirement Text File Run

In [41]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'



# Data
---



In [42]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import tensorflow as tf
from sklearn.decomposition import PCA
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array


pca32 = ['pca32-1', 'pca32-2', 'pca32-3', 'pca32-4']
autoencoder32 = ['autoencoder32-1', 'autoencoder32-2', 'autoencoder32-3', 'autoencoder32-4']
pca30 = ['pca30-1', 'pca30-2', 'pca30-3', 'pca30-4']
autoencoder30 = ['autoencoder30-1', 'autoencoder30-2', 'autoencoder30-3', 'autoencoder30-4']
pca16 = ['pca16-1', 'pca16-2', 'pca16-3', 'pca16-4', 'pca16-compact']
autoencoder16 = ['autoencoder16-1', 'autoencoder16-2', 'autoencoder16-3', 'autoencoder16-4', 'autoencoder16-compact']
pca12 = ['pca12-1', 'pca12-2', 'pca12-3', 'pca12-4']
autoencoder12 = ['autoencoder12-1', 'autoencoder12-2', 'autoencoder12-3', 'autoencoder12-4']
pca60= ['pca60-1','pca60-2','pca60-3','pca60-4']
autoencoder60 = ['autoencoder60-1','autoencoder60-2','autoencoder60-3','autoencoder60-4']
pca80 = ['pca80-1', 'pca80-2', 'pca80-3', 'pca80-4']
autoencoder80 = ['autoencoder80-1', 'autoencoder80-2', 'autoencoder80-3', 'autoencoder80-4']

from tensorflow.keras.utils import load_img, img_to_array

def data_load_and_process(dataset, classes=[0, 1], feature_reduction='resize256', binary=True):
    # Initialize lists for training data
    X_train = []
    Y_train = []

    # Define paths for training data
    capuchinbird_dir = os.path.join(dataset, 'Train', 'Parsed_Capuchinbird_Clips')
    not_capuchinbird_dir = os.path.join(dataset, 'Train', 'Parsed_Not_Capuchinbird_Clips')

    # Load Capuchinbird images for training
    for filename in os.listdir(capuchinbird_dir):
        if filename.endswith('.png'):
            img_path = os.path.join(capuchinbird_dir, filename)
            img = load_img(img_path, target_size=(370, 969), color_mode='grayscale')  # Force grayscale
            img_array = img_to_array(img)  # Convert to array
            X_train.append(img_array)
            Y_train.append(1)  # Label for Capuchinbird

    # Load Not_Capuchinbird images for training
    for filename in os.listdir(not_capuchinbird_dir):
        if filename.endswith('.png'):
            img_path = os.path.join(not_capuchinbird_dir, filename)
            img = load_img(img_path, target_size=(370, 969), color_mode='grayscale')  # Force grayscale
            img_array = img_to_array(img)  # Convert to array
            X_train.append(img_array)
            Y_train.append(0)  # Label for Not_Capuchinbird

    # Convert lists to numpy arrays
    X_train = np.array(X_train)
    Y_train = np.array(Y_train)

    # Repeat the same for test data
    X_test = []
    Y_test = []
    capuchinbird_test_dir = os.path.join(dataset, 'Test', 'Parsed_Capuchinbird_Clips')
    not_capuchinbird_test_dir = os.path.join(dataset, 'Test', 'Parsed_Not_Capuchinbird_Clips')

    for filename in os.listdir(capuchinbird_test_dir):
        if filename.endswith('.png'):
            img_path = os.path.join(capuchinbird_test_dir, filename)
            img = load_img(img_path, target_size=(370, 969), color_mode='grayscale')  # Force grayscale
            img_array = img_to_array(img)  # Convert to array
            X_test.append(img_array)
            Y_test.append(1)  # Label for Capuchinbird

    for filename in os.listdir(not_capuchinbird_test_dir):
        if filename.endswith('.png'):
            img_path = os.path.join(not_capuchinbird_test_dir, filename)
            img = load_img(img_path, target_size=(370, 969), color_mode='grayscale')  # Force grayscale
            img_array = img_to_array(img)  # Convert to array
            X_test.append(img_array)
            Y_test.append(0)  # Label for Not_Capuchinbird

    # Convert lists to numpy arrays
    X_test = np.array(X_test)
    Y_test = np.array(Y_test)

    # Check shapes of loaded data
    print(f'X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}')
    print(f'X_test shape: {X_test.shape}, Y_test shape: {Y_test.shape}')


    print("Normalized")
    X_train = X_train / 255.0  # Normalize pixel values to [0, 1]
    X_test = X_test / 255.0

    print(f"Loaded X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")
    print(f"Loaded X_test shape: {X_test.shape}, Y_test shape: {Y_test.shape}")


    #binary classification
    if binary:
        Y_train = [1 if y == classes[1] else -1 for y in Y_train]  # Convert class '1' to 1 and '0' to -1
        Y_test = [1 if y == classes[1] else -1 for y in Y_test]




    if feature_reduction == 'resize256':
        X_train = tf.image.resize(X_train[:], (256,1)).numpy()
        X_test = tf.image.resize(X_test[:], (256,1)).numpy()
        X_train, X_test = tf.squeeze(X_train).numpy(), tf.squeeze(X_test).numpy()
        return X_train, X_test, Y_train, Y_test


    elif feature_reduction == 'pca8' or feature_reduction in pca32 \
            or feature_reduction in pca30 or feature_reduction in pca16 or feature_reduction in pca12 or feature_reduction in pca60 or feature_reduction in pca80:

        X_train = tf.image.resize(X_train[:], (784, 1)).numpy()
        X_test = tf.image.resize(X_test[:], (784, 1)).numpy()
        X_train, X_test = tf.squeeze(X_train), tf.squeeze(X_test)

        if feature_reduction == 'pca8':
            pca = PCA(8)
        elif feature_reduction in pca32:
            pca = PCA(32)
        elif feature_reduction in pca30:
            pca = PCA(30)
        elif feature_reduction in pca16:
            pca = PCA(16)
        elif feature_reduction in pca12:
            pca = PCA(12)
        elif feature_reduction in pca60:
            pca= PCA(60)
        elif feature_reduction in pca80:
            pca=PCA(80)


        X_train = pca.fit_transform(X_train)
        X_test = pca.transform(X_test)

        # Rescale for angle embedding
        if feature_reduction == 'pca8' or feature_reduction == 'pca16-compact' or \
                feature_reduction in pca30 or feature_reduction in pca12: #or feature_reduction in pca128
            X_train, X_test = (X_train - X_train.min()) * (np.pi / (X_train.max() - X_train.min())),\
                              (X_test - X_test.min()) * (np.pi / (X_test.max() - X_test.min()))
        return X_train, X_test, Y_train, Y_test

    elif feature_reduction == 'autoencoder8' or feature_reduction in autoencoder32 \
            or feature_reduction in autoencoder30 or feature_reduction in autoencoder16 or feature_reduction in autoencoder12 or feature_reduction in autoencoder60 or feature_reduction in autoencoder80:
        if feature_reduction == 'autoencoder8':
            latent_dim = 8
        elif feature_reduction in autoencoder32:
            latent_dim = 32
        elif feature_reduction in autoencoder30:
            latent_dim = 30
        elif feature_reduction in autoencoder16:
            latent_dim = 16
        elif feature_reduction in autoencoder12:
            latent_dim = 12
        elif feature_reduction in autoencoder60:
            latent_dim = 60
        elif feature_reduction in autoencoder80:
            latent_dim=80


        
    class Autoencoder(Model):
            def __init__(self, latent_dim):
                super(Autoencoder, self).__init__()
                self.latent_dim = latent_dim
                self.encoder = tf.keras.Sequential([
                    layers.Flatten(),
                    layers.Dense(latent_dim, activation='relu'),
                ])
                self.decoder = tf.keras.Sequential([
                    layers.Dense(358530, activation='sigmoid'),
                    layers.Reshape((370, 969))
                ])

            def call(self, x):
                encoded = self.encoder(x)
                decoded = self.decoder(encoded)
                return decoded

    autoencoder = Autoencoder(latent_dim)

    autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())
    autoencoder.fit(X_train, X_train,
                        epochs=10,
                        shuffle=True,
                        validation_data=(X_test, X_test))

    X_train, X_test = autoencoder.encoder(X_train).numpy(), autoencoder.encoder(X_test).numpy()

    # Rescale for Angle Embedding
    # Note this is not a rigorous rescaling method
    if feature_reduction == 'autoencoder8' or feature_reduction == 'autoencoder16-compact' or\
                feature_reduction in autoencoder30 or feature_reduction in autoencoder12:
            X_train, X_test = (X_train - X_train.min()) * (np.pi / (X_train.max() - X_train.min())), \
                              (X_test - X_test.min()) * (np.pi / (X_test.max() - X_test.min()))

    return X_train, X_test, Y_train, Y_test

#Unitary
---


In [43]:
# This module contains the set of unitary ansatze that will be used to benchmark the performances of Quantum Convolutional Neural Network (QCNN) in QCNN.ipynb module
import pennylane as qml

# Unitary Ansatze for Convolutional Layer
def U_TTN(params, wires):  # 2 params cc_1
    qml.RY(params[0], wires=wires[0])
    qml.RY(params[1], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])


def U_5(params, wires):  # 10 params CC_7
    qml.RX(params[0], wires=wires[0])
    qml.RX(params[1], wires=wires[1])
    qml.RZ(params[2], wires=wires[0])
    qml.RZ(params[3], wires=wires[1])
    qml.CRZ(params[4], wires=[wires[1], wires[0]])
    qml.CRZ(params[5], wires=[wires[0], wires[1]])
    qml.RX(params[6], wires=wires[0])
    qml.RX(params[7], wires=wires[1])
    qml.RZ(params[8], wires=wires[0])
    qml.RZ(params[9], wires=wires[1])


def U_6(params, wires):  # 10 params CC_8
    qml.RX(params[0], wires=wires[0])
    qml.RX(params[1], wires=wires[1])
    qml.RZ(params[2], wires=wires[0])
    qml.RZ(params[3], wires=wires[1])
    qml.CRX(params[4], wires=[wires[1], wires[0]])
    qml.CRX(params[5], wires=[wires[0], wires[1]])
    qml.RX(params[6], wires=wires[0])
    qml.RX(params[7], wires=wires[1])
    qml.RZ(params[8], wires=wires[0])
    qml.RZ(params[9], wires=wires[1])


def U_9(params, wires):  # 2 params cc_2
    qml.Hadamard(wires=wires[0])
    qml.Hadamard(wires=wires[1])
    qml.CZ(wires=[wires[0], wires[1]])
    qml.RX(params[0], wires=wires[0])
    qml.RX(params[1], wires=wires[1])


def U_13(params, wires):  # 6 params cc_4
    qml.RY(params[0], wires=wires[0])
    qml.RY(params[1], wires=wires[1])
    qml.CRZ(params[2], wires=[wires[1], wires[0]])
    qml.RY(params[3], wires=wires[0])
    qml.RY(params[4], wires=wires[1])
    qml.CRZ(params[5], wires=[wires[0], wires[1]])


def U_14(params, wires):  # 6 params cc_5
    qml.RY(params[0], wires=wires[0])
    qml.RY(params[1], wires=wires[1])
    qml.CRX(params[2], wires=[wires[1], wires[0]])
    qml.RY(params[3], wires=wires[0])
    qml.RY(params[4], wires=wires[1])
    qml.CRX(params[5], wires=[wires[0], wires[1]])


def U_15(params, wires):  # 4 params cc_3
    qml.RY(params[0], wires=wires[0])
    qml.RY(params[1], wires=wires[1])
    qml.CNOT(wires=[wires[1], wires[0]])
    qml.RY(params[2], wires=wires[0])
    qml.RY(params[3], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])


def U_SO4(params, wires):  # 6 params CC_6
    qml.RY(params[0], wires=wires[0])
    qml.RY(params[1], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.RY(params[2], wires=wires[0])
    qml.RY(params[3], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.RY(params[4], wires=wires[0])
    qml.RY(params[5], wires=wires[1])


def U_SU4(params, wires): # 15 params CC_9
    qml.U3(params[0], params[1], params[2], wires=wires[0])
    qml.U3(params[3], params[4], params[5], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.RY(params[6], wires=wires[0])
    qml.RZ(params[7], wires=wires[1])
    qml.CNOT(wires=[wires[1], wires[0]])
    qml.RY(params[8], wires=wires[0])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.U3(params[9], params[10], params[11], wires=wires[0])
    qml.U3(params[12], params[13], params[14], wires=wires[1])




# Pooling Layer



def custom_Pooling1(params, wires): #10 params
    if len(params) < 10:  # Adjust based on your needs
        raise ValueError("Params must have at least 10 elements")
    
    # Apply gates to the first four qubits
    qml.Hadamard(wires=wires[0])
    qml.RZ(params[0], wires=wires[0])

    qml.Hadamard(wires=wires[1])
    qml.RZ(params[1], wires=wires[1])

    qml.Hadamard(wires=wires[2])
    qml.RZ(params[2], wires=wires[2])

    qml.Hadamard(wires=wires[3])
    qml.RZ(params[3], wires=wires[3])

    # Apply ZZ gates to the first four qubits
    qml.IsingZZ(params[4], wires=[wires[0], wires[1]])
    qml.IsingZZ(params[5], wires=[wires[0], wires[2]])
    qml.IsingZZ(params[6], wires=[wires[0], wires[3]])
    qml.IsingZZ(params[7], wires=[wires[1], wires[2]])
    qml.IsingZZ(params[8], wires=[wires[1], wires[3]])
    qml.IsingZZ(params[9], wires=[wires[2], wires[3]])

    # Apply controlled-Y gates
    qml.CY(wires=[wires[0], wires[1]]) 
    qml.CY(wires=[wires[0], wires[2]]) 
    qml.CY(wires=[wires[0], wires[3]]) 

    # Apply controlled-Y gates
    qml.CY(wires=[wires[1], wires[2]]) 
    qml.CY(wires=[wires[1], wires[3]]) 
    qml.CY(wires=[wires[2], wires[3]])

#Can't use it as it takes 5 qubits
def custom_Pooling2(params, wires): #14 parameters, 5 wires
    if len(params) < 14:  # Adjust based on your needs
        raise ValueError("Params must have at least 14 elements")

    # Apply gates to the first four qubits
    qml.Hadamard(wires=wires[0])
    qml.RZ(params[0], wires=wires[0])

    qml.Hadamard(wires=wires[1])
    qml.RZ(params[1], wires=wires[1])

    qml.Hadamard(wires=wires[2])
    qml.RZ(params[2], wires=wires[2])

    qml.Hadamard(wires=wires[3])
    qml.RZ(params[3], wires=wires[3])

    qml.Hadamard(wires=wires[4])

    # Apply ZZ gates to the first four qubits
    qml.IsingZZ(params[4], wires=[wires[0], wires[1]])
    qml.IsingZZ(params[5], wires=[wires[0], wires[2]])
    qml.IsingZZ(params[6], wires=[wires[0], wires[3]])
    qml.IsingZZ(params[7], wires=[wires[1], wires[2]])
    qml.IsingZZ(params[8], wires=[wires[1], wires[3]])
    qml.IsingZZ(params[9], wires=[wires[2], wires[3]])

    # Apply RX gates to the second half of qubits
    qml.RX(params[10], wires=wires[0])
    qml.RX(params[11], wires=wires[1])
    qml.RX(params[12], wires=wires[2])
    qml.RX(params[13], wires=wires[3])

    
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.CNOT(wires=[wires[1], wires[2]])
    qml.CNOT(wires=[wires[2], wires[3]])
    qml.CNOT(wires=[wires[3], wires[0]])

    qml.CY(wires=[wires[0], wires[4]])
    qml.CY(wires=[wires[1], wires[4]])
    qml.CY(wires=[wires[2], wires[4]])
    qml.CY(wires=[wires[3], wires[4]])
 
    qml.Hadamard(wires=wires[4])




def mod_circuit_a(params, wires): #2 params

    qml.CRZ(params[0], wires=[wires[0], wires[1]]) 
    qml.X(wires=wires[0])
    qml.CRX(params[1], wires=[wires[0], wires[1]])

def mod_circuit_b(params, wires): #4 paarams
    qml.RY(params[0], wires=wires[0])  # RY rotation on wire 0
    qml.RY(params[1], wires=wires[1])  # RY rotation on wire 1
    qml.CNOT(wires=[wires[0], wires[1]])  # CNOT with control wire 0 and target wire 1
    qml.CRZ(params[2], wires=[wires[0], wires[1]])  # Controlled RZ gate
    qml.X(wires=wires[0])  # Pauli-X gate on wire 0
    qml.CRX(params[3], wires=[wires[0], wires[1]])  # Controlled RX gate


def mod_circuit_c(params, wires): #4params
    qml.Hadamard(wires=wires[0])  # Hadamard gate on wire 0
    qml.Hadamard(wires=wires[1])  # Hadamard gate on wire 1
    qml.CZ(wires=[wires[0], wires[1]])  # Controlled-Z gate
    qml.RX(params[0], wires=wires[0])  # RX rotation on wire 0
    qml.RX(params[1], wires=wires[1])  # RX rotation on wire 1
    qml.CRZ(params[2], wires=[wires[0], wires[1]])  # Controlled RZ gate
    qml.X(wires=wires[0])  # Pauli-X gate on wire 0
    qml.CRX(params[3], wires=[wires[0], wires[1]])  # Controlled RX gate

def custom_Pooling3_Mod_b(params, wires): #22 parameters, 4 wires
    if len(params) < 22:  # Adjust based on your needs
        raise ValueError("Params must have at least 22 elements")
    # Apply gates to the first four qubits
    qml.Hadamard(wires=wires[0])
    qml.RZ(params[0], wires=wires[0])

    qml.Hadamard(wires=wires[1])
    qml.RZ(params[1], wires=wires[1])

    qml.Hadamard(wires=wires[2])
    qml.RZ(params[2], wires=wires[2])

    qml.Hadamard(wires=wires[3])
    qml.RZ(params[3], wires=wires[3])

    # Apply ZZ gates to the first four qubits

    qml.IsingZZ(params[4], wires=[wires[0], wires[1]])
    qml.IsingZZ(params[5], wires=[wires[0], wires[2]])
    qml.IsingZZ(params[6], wires=[wires[0], wires[3]])
    qml.IsingZZ(params[7], wires=[wires[1], wires[2]])
    qml.IsingZZ(params[8], wires=[wires[1], wires[3]])
    qml.IsingZZ(params[9], wires=[wires[2], wires[3]])


    # Apply MOD gates (using mod_circuit_b)
    mod_circuit_b([params[10], params[11], params[12], params[13]], wires=[wires[0], wires[1]])  # MOD gate for qubits 0 and 1
    mod_circuit_b([params[14], params[15], params[16], params[17]], wires=[wires[2], wires[3]])  # MOD gate for qubits 2 and 3
    mod_circuit_b([params[18], params[19], params[20], params[21]], wires=[wires[1], wires[3]])  # MOD gate for qubits 1 and 3 



def custom_ansatz3_Mod_c(params, wires): # 22 parameters, 4 wires
    if len(params) < 22:  # Adjust based on your needs
        raise ValueError("Params must have at least 22 elements")
    # Apply gates to the first four qubits
    qml.Hadamard(wires=wires[0])
    qml.RZ(params[0], wires=wires[0])

    qml.Hadamard(wires=wires[1])
    qml.RZ(params[1], wires=wires[1])

    qml.Hadamard(wires=wires[2])
    qml.RZ(params[2], wires=wires[2])

    qml.Hadamard(wires=wires[3])
    qml.RZ(params[3], wires=wires[3])

    # Apply ZZ gates to the first four qubits
    qml.IsingZZ(params[4], wires=[wires[0], wires[1]])
    qml.IsingZZ(params[5], wires=[wires[0], wires[2]])
    qml.IsingZZ(params[6], wires=[wires[0], wires[3]])
    qml.IsingZZ(params[7], wires=[wires[1], wires[2]])
    qml.IsingZZ(params[8], wires=[wires[1], wires[3]])
    qml.IsingZZ(params[9], wires=[wires[2], wires[3]])

    # Apply MOD gates (using mod_circuit_c)
    mod_circuit_c([params[10], params[11], params[12], params[13]], wires=[wires[0], wires[1]])  # MOD gate for qubits 0 and 1
    mod_circuit_c([params[14], params[15], params[16], params[17]], wires=[wires[2], wires[3]])  # MOD gate for qubits 2 and 3
    mod_circuit_c([params[18], params[19], params[20], params[21]], wires=[wires[1], wires[3]])  # MOD gate for qubits 1 and 3


def custom_ansatz3_c_LessParam(params, wires): # 12 parameters, 4 wires
    if len(params) < 12:  # Adjust based on your needs
        raise ValueError("Params must have at least 12 elements")
    # Apply gates to the first four qubits
    qml.Hadamard(wires=wires[0])
    qml.RZ(params[0], wires=wires[0])

    qml.Hadamard(wires=wires[1])
    qml.RZ(params[1], wires=wires[1])

    qml.Hadamard(wires=wires[2])
    qml.RZ(params[2], wires=wires[2])

    qml.Hadamard(wires=wires[3])
    qml.RZ(params[3], wires=wires[3])

    # Apply ZZ gates to the first four qubits
    qml.IsingZZ(params[4], wires=[wires[0], wires[1]])
    qml.IsingZZ(params[5], wires=[wires[0], wires[2]])
    qml.IsingZZ(params[6], wires=[wires[0], wires[3]])
    qml.IsingZZ(params[7], wires=[wires[1], wires[2]])
    qml.IsingZZ(params[8], wires=[wires[1], wires[3]])
    qml.IsingZZ(params[9], wires=[wires[2], wires[3]])

    # Apply MOD gates (using mod_circuit_c)
    mod_circuit_c([params[10], params[11], params[4], params[5]], wires=[wires[0], wires[1]])  # MOD gate for qubits 0 and 1
    mod_circuit_c([params[6], params[7], params[8], params[9]], wires=[wires[2], wires[3]])  # MOD gate for qubits 2 and 3
    mod_circuit_c([params[10], params[11], params[6], params[7]], wires=[wires[1], wires[3]])  # MOD gate for qubits 1 and 3

def custom_Pooling3_Mod_a(params, wires): # 16 parameters, 4 wires
    # Apply gates to the first four qubits
    qml.Hadamard(wires=wires[0])
    qml.RZ(params[0], wires=wires[0])

    qml.Hadamard(wires=wires[1])
    qml.RZ(params[1], wires=wires[1])

    qml.Hadamard(wires=wires[2])
    qml.RZ(params[2], wires=wires[2])

    qml.Hadamard(wires=wires[3])
    qml.RZ(params[3], wires=wires[3])

    # Apply ZZ gates to the first four qubits
    qml.IsingZZ(params[4], wires=[wires[0], wires[1]])
    qml.IsingZZ(params[5], wires=[wires[0], wires[2]])
    qml.IsingZZ(params[6], wires=[wires[0], wires[3]])
    qml.IsingZZ(params[7], wires=[wires[1], wires[2]])
    qml.IsingZZ(params[8], wires=[wires[1], wires[3]])
    qml.IsingZZ(params[9], wires=[wires[2], wires[3]])

    # Apply MOD gates (using mod_circuit_a)
    mod_circuit_a([params[10], params[11]], wires=[wires[0], wires[1]])  # MOD gate for qubits 0 and 1
    mod_circuit_a([params[12], params[13]], wires=[wires[2], wires[3]])  # MOD gate for qubits 2 and 3
    mod_circuit_a([params[14], params[15]], wires=[wires[1], wires[3]])  # MOD gate for qubits 1 and 3


def custom_Pooling4(params, wires): #14 parameters
    if len(params) < 14:  # Adjust based on your needs
        raise ValueError("Params must have at least 14 elements")

    # Apply gates to the first four qubits
    qml.Hadamard(wires=wires[0])
    qml.RZ(params[0], wires=wires[0])

    qml.Hadamard(wires=wires[1])
    qml.RZ(params[1], wires=wires[1])

    qml.Hadamard(wires=wires[2])
    qml.RZ(params[2], wires=wires[2])

    qml.Hadamard(wires=wires[3])
    qml.RZ(params[3], wires=wires[3])

    # Apply ZZ gates to the first four qubits
    qml.IsingZZ(params[4], wires=[wires[0], wires[1]])
    qml.IsingZZ(params[5], wires=[wires[0], wires[2]])
    qml.IsingZZ(params[6], wires=[wires[0], wires[3]])
    qml.IsingZZ(params[7], wires=[wires[1], wires[2]])
    qml.IsingZZ(params[8], wires=[wires[1], wires[3]])
    qml.IsingZZ(params[9], wires=[wires[2], wires[3]])

    # Apply RX gates to the second half of qubits
    qml.RX(params[10], wires=wires[0])
    qml.RX(params[11], wires=wires[1])
    qml.RX(params[12], wires=wires[2])
    qml.RX(params[13], wires=wires[3])

    
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.CNOT(wires=[wires[1], wires[2]])
    qml.CNOT(wires=[wires[2], wires[3]])
    qml.CNOT(wires=[wires[3], wires[0]])






def Pooling_ansatz1(params, wires): #2 params
    qml.CRZ(params[0], wires=[wires[0], wires[1]])
    qml.PauliX(wires=wires[0])
    qml.CRX(params[1], wires=[wires[0], wires[1]])

def Pooling_ansatz2(wires): #0 params
    qml.CRZ(wires=[wires[0], wires[1]])

def Pooling_ansatz3(*params, wires): #3 params
    qml.CRot(*params, wires=[wires[0], wires[1]])


# Angular Hybrid

---

In [44]:
# This is an implementation of an alternative Mottonen State Preparation to avoid normalization problem.
import pennylane as qml

# 3 bits of information is embedded in 2 wires
def Angular_Hybrid_2(X, wires):
    qml.RY(X[0], wires=wires[0])

    qml.PauliX(wires=wires[0])
    qml.CRY(X[1], wires=[wires[0], wires[1]])
    qml.PauliX(wires=wires[0])
    qml.CRY(X[2], wires=[wires[0], wires[1]])

# 15 bits of information is embedded in 4 wires
def Angular_Hybrid_4(X, wires):
    qml.RY(X[0], wires=wires[0])

    qml.PauliX(wires=wires[0])
    qml.CRY(X[1], wires=[wires[0], wires[1]])
    qml.PauliX(wires=wires[0])
    qml.CRY(X[2], wires=[wires[0], wires[1]])

    qml.RY(X[3], wires=wires[2])
    qml.CNOT(wires=[wires[1], wires[2]])
    qml.RY(X[4], wires=wires[2])
    qml.CNOT(wires=[wires[0], wires[2]])
    qml.RY(X[5], wires=wires[2])
    qml.CNOT(wires=[wires[1], wires[2]])
    qml.RY(X[6], wires=wires[2])
    qml.CNOT(wires=[wires[0], wires[2]])

    qml.RY(X[7], wires=wires[3])
    qml.CNOT(wires=[wires[2], wires[3]])
    qml.RY(X[8], wires=wires[3])
    qml.CNOT(wires=[wires[1], wires[3]])
    qml.RY(X[9], wires=wires[3])
    qml.CNOT(wires=[wires[2], wires[3]])
    qml.RY(X[10], wires=wires[3])
    qml.CNOT(wires=[wires[0], wires[3]])
    qml.RY(X[11], wires=wires[3])
    qml.CNOT(wires=[wires[2], wires[3]])
    qml.RY(X[12], wires=wires[3])
    qml.CNOT(wires=[wires[1], wires[3]])
    qml.RY(X[13], wires=wires[3])
    qml.CNOT(wires=[wires[2], wires[3]])
    qml.RY(X[14], wires=wires[3])
    qml.CNOT(wires=[wires[0], wires[3]])

# Embedding

---

In [45]:
# This is an implementation of data_embedding function used for 8 qubits Quantum Convolutional Neural Network (QCNN)
# and Hierarchical Quantum Classifier circuit.
import pennylane as qml
from pennylane.templates.embeddings import AmplitudeEmbedding, AngleEmbedding
from pennylane.templates.state_preparations import MottonenStatePreparation
import numpy as np

def data_embedding(X, embedding_type='Amplitude'):
    if embedding_type == 'Amplitude':
        AmplitudeEmbedding(X, wires=range(8), normalize=True)
    elif embedding_type == 'Angle':
        AngleEmbedding(X, wires=range(8), rotation='Y')
    elif embedding_type == 'Angle-compact':
        AngleEmbedding(X[:8], wires=range(8), rotation='X')
        AngleEmbedding(X[8:16], wires=range(8), rotation='Y')

    # Hybrid Direct Embedding (HDE)
    elif embedding_type == 'Amplitude-Hybrid4-1' or embedding_type == 'Amplitude-Hybrid4-2' or \
        embedding_type == 'Amplitude-Hybrid4-3' or embedding_type == 'Amplitude-Hybrid4-4':
        X1 = X[:2 ** 4]
        X2 = X[2 ** 4:2 ** 5]
        
        # Check for nan values
        if np.isnan(X1).any() or np.isnan(X2).any():
            raise ValueError("Input vectors contain nan values.")
        
        # Calculate norms
        norm_X1 = np.linalg.norm(X1)
        norm_X2 = np.linalg.norm(X2)
        
        # Handle zero norms
        if norm_X1 == 0:
            X1 = np.ones_like(X1)  # Replace with a valid vector if norm is zero
            norm_X1 = np.linalg.norm(X1)
        if norm_X2 == 0:
            X2 = np.ones_like(X2)  # Replace with a valid vector if norm is zero
            norm_X2 = np.linalg.norm(X2)
        
        # Normalize X1 and X2
        X1, X2 = X1 / norm_X1, X2 / norm_X2

        if np.isnan(X1).any() or np.isnan(X2).any():
            raise ValueError("Normalization resulted in nan values.")
    
    # Check if normalization resulted in nan values
    if np.isnan(X1).any() or np.isnan(X2).any():
        raise ValueError("Normalization resulted in nan values.")
    
    if embedding_type == 'Amplitude-Hybrid4-1':
        MottonenStatePreparation(X1, wires=[0, 1, 2, 3])
        MottonenStatePreparation(X2, wires=[4, 5, 6, 7])
    elif embedding_type == 'Amplitude-Hybrid4-2':
        MottonenStatePreparation(X1, wires=[0, 2, 4, 6])
        MottonenStatePreparation(X2, wires=[1, 3, 5, 7])
    elif embedding_type == 'Amplitude-Hybrid4-3':
        MottonenStatePreparation(X1, wires=[0, 1, 6, 7])
        MottonenStatePreparation(X2, wires=[2, 3, 4, 5])
    elif embedding_type == 'Amplitude-Hybrid4-4':
        MottonenStatePreparation(X1, wires=[0, 3, 4, 7])
        MottonenStatePreparation(X2, wires=[1, 2, 5, 6])


        if embedding_type == 'Amplitude-Hybrid4-1':
            MottonenStatePreparation(X1, wires=[0, 1, 2, 3])
            MottonenStatePreparation(X2, wires=[4, 5, 6, 7])
        elif embedding_type == 'Amplitude-Hybrid4-2':
            MottonenStatePreparation(X1, wires=[0, 2, 4, 6])
            MottonenStatePreparation(X2, wires=[1, 3, 5, 7])
        elif embedding_type == 'Amplitude-Hybrid4-3':
            MottonenStatePreparation(X1, wires=[0, 1, 6, 7])
            MottonenStatePreparation(X2, wires=[2, 3, 4, 5])
        elif embedding_type == 'Amplitude-Hybrid4-4':
            MottonenStatePreparation(X1, wires=[0, 3, 4, 7])
            MottonenStatePreparation(X2, wires=[1, 2, 5, 6])

    elif embedding_type == 'Amplitude-Hybrid2-1' or embedding_type == 'Amplitude-Hybrid2-2' \
            or embedding_type == 'Amplitude-Hybrid2-3' or embedding_type == 'Amplitude-Hybrid2-4':
        X1 = X[:4]
        X2 = X[4:8]
        X3 = X[8:12]
        X4 = X[12:16]
        norm_X1, norm_X2, norm_X3, norm_X4 = np.linalg.norm(X1), np.linalg.norm(X2), np.linalg.norm(X3), np.linalg.norm(
            X4)
        X1, X2, X3, X4 = X1 / norm_X1, X2 / norm_X2, X3 / norm_X3, X4 / norm_X4

        if embedding_type == 'Amplitude-Hybrid2-1':
            MottonenStatePreparation(X1, wires=[0,1])
            MottonenStatePreparation(X2, wires=[2,3])
            MottonenStatePreparation(X3, wires=[4,5])
            MottonenStatePreparation(X4, wires=[6,7])
        elif embedding_type == 'Amplitude-Hybrid2-2':
            MottonenStatePreparation(X1, wires=[0,4])
            MottonenStatePreparation(X2, wires=[1,5])
            MottonenStatePreparation(X3, wires=[2,6])
            MottonenStatePreparation(X4, wires=[3,7])
        elif embedding_type == 'Amplitude-Hybrid2-3':
            MottonenStatePreparation(X1, wires=[0,7])
            MottonenStatePreparation(X2, wires=[1,6])
            MottonenStatePreparation(X3, wires=[2,5])
            MottonenStatePreparation(X4, wires=[3,4])
        elif embedding_type == 'Amplitude-Hybrid2-4':
            MottonenStatePreparation(X1, wires=[0,2])
            MottonenStatePreparation(X2, wires=[1,3])
            MottonenStatePreparation(X3, wires=[4,6])
            MottonenStatePreparation(X4, wires=[5,7])

    # Hybrid Angle Embedding (HAE)
    elif embedding_type == 'Angular-Hybrid4-1' or embedding_type == 'Angular-Hybrid4-2' or \
            embedding_type == 'Angular-Hybrid4-3' or embedding_type == 'Angular-Hybrid4-4':
        N = 15 # 15 classical data in 4 qubits
        X1 = X[:N]
        X2 = X[N:2*N]

        if embedding_type == 'Angular-Hybrid4-1':
            Angular_Hybrid_4(X1, wires=[0, 1, 2, 3])
            Angular_Hybrid_4(X2, wires=[4, 5, 6, 7])
        elif embedding_type == 'Angular-Hybrid4-2':
            Angular_Hybrid_4(X1, wires=[0, 2, 4, 6])
            Angular_Hybrid_4(X2, wires=[1, 3, 5, 7])
        elif embedding_type == 'Angular-Hybrid4-3':
            Angular_Hybrid_4(X1, wires=[0, 1, 6, 7])
            Angular_Hybrid_4(X2, wires=[2, 3, 4, 5])
        elif embedding_type == 'Angular-Hybrid4-4':
            Angular_Hybrid_4(X1, wires=[0, 3, 4, 7])
            Angular_Hybrid_4(X2, wires=[1, 2, 5, 6])

    elif embedding_type == 'Angular-Hybrid2-1' or embedding_type == 'Angular-Hybrid2-2' \
            or embedding_type == 'Angular-Hybrid2-3' or embedding_type == 'Angular-Hybrid2-4':
        N = 3  # 3 classical bits in 2 qubits
        X1 = X[:N]
        X2 = X[N:2*N]
        X3 = X[2*N:3*N]
        X4 = X[3*N:4*N]

        if embedding_type == 'Angular-Hybrid2-1':
            Angular_Hybrid_2(X1, wires=[0,1])
            Angular_Hybrid_2(X2, wires=[2,3])
            Angular_Hybrid_2(X3, wires=[4,5])
            Angular_Hybrid_2(X4, wires=[6,7])
        elif embedding_type == 'Angular-Hybrid2-2':
            Angular_Hybrid_2(X1, wires=[0,4])
            Angular_Hybrid_2(X2, wires=[1,5])
            Angular_Hybrid_2(X3, wires=[2,6])
            Angular_Hybrid_2(X4, wires=[3,7])
        elif embedding_type == 'Angular-Hybrid2-3':
            Angular_Hybrid_2(X1, wires=[0,7])
            Angular_Hybrid_2(X2, wires=[1,6])
            Angular_Hybrid_2(X3, wires=[2,5])
            Angular_Hybrid_2(X4, wires=[3,4])
        elif embedding_type == 'Angular-Hybrid2-4':
            Angular_Hybrid_2(X1, wires=[0,2])
            Angular_Hybrid_2(X2, wires=[1,3])
            Angular_Hybrid_2(X3, wires=[4,6])
            Angular_Hybrid_2(X4, wires=[5,7])







# QCNN Circuit

In [46]:
import pennylane as qml
# import unitary
# import embedding

# Quantum Circuits for Convolutional layers
def conv_layer1(U, params):
    
    """U(params, wires=[0, 7]) # we have 8 wires
    for i in range(0, 8, 2):
        U(params, wires=[i, i + 1]) #wire 0 and 1, wire 2 and 3, wire 4 and 5, wire 6 and 7
    for i in range(1, 7, 2):
        U(params, wires=[i, i + 1]) #wire 1 and 2, wire 3 and 4, wire 5 and 6"""
    if U == 'U_TTN':
        U_TTN(params, wires=[0, 7])
        for i in range(0, 8, 2):
            U_TTN(params, wires=[i, i + 1])
        for i in range(1, 7, 2):
            U_TTN(params, wires=[i, i + 1])
    elif U == 'U_5':
        U_5(params, wires=[0, 7])
        for i in range(0, 8, 2):
            U_5(params, wires=[i, i + 1])
        for i in range(1, 7, 2):
            U_5(params, wires=[i, i + 1])
    elif U == 'U_6':
        U_6(params, wires=[0, 7])
        for i in range(0, 8, 2):
            U_6(params, wires=[i, i + 1])
        for i in range(1, 7, 2):
            U_6(params, wires=[i, i + 1])
    elif U == 'U_9':
        U_9(params, wires=[0, 7])
        for i in range(0, 8, 2):
            U_9(params, wires=[i, i + 1])
        for i in range(1, 7, 2):
            U_9(params, wires=[i, i + 1])
    elif U == 'U_13':
        U_13(params, wires=[0, 7])
        for i in range(0, 8, 2):
            U_13(params, wires=[i, i + 1])
        for i in range(1, 7, 2):
            U_13(params, wires=[i, i + 1])
    elif U == 'U_14':
        U_14(params, wires=[0, 7])
        for i in range(0, 8, 2):
            U_14(params, wires=[i, i + 1])
        for i in range(1, 7, 2):
            U_14(params, wires=[i, i + 1])
    elif U == 'U_15':
        U_15(params, wires=[0, 7])
        for i in range(0, 8, 2):
            U_15(params, wires=[i, i + 1])
        for i in range(1, 7, 2):
            U_15(params, wires=[i, i + 1])
    elif U == 'U_SO4':
        U_SO4(params, wires=[0, 7])
        for i in range(0, 8, 2):
            U_SO4(params, wires=[i, i + 1])
        for i in range(1, 7, 2):
            U_SO4(params, wires=[i, i + 1])
    elif U == 'U_SU4':
        U_SU4(params, wires=[0, 7])
        for i in range(0, 8, 2):
            U_SU4(params, wires=[i, i + 1])
        for i in range(1, 7, 2):
            U_SU4(params, wires=[i, i + 1])
    elif U == 'U_SU4_no_pooling':
        U_SU4(params, wires=[0, 7])
        for i in range(0, 8, 2):
            U_SU4(params, wires=[i, i + 1])
        for i in range(1, 7, 2):
            U_SU4(params, wires=[i, i + 1])
    elif U == 'U_SU4_1D':
        U_SU4(params, wires=[0, 7])
        for i in range(0, 8, 2):
            U_SU4(params, wires=[i, i + 1])
        for i in range(1, 7, 2):
            U_SU4(params, wires=[i, i + 1])
    elif U == 'U_9_1D':
        U_9(params, wires=[0, 7])
        for i in range(0, 8, 2):
            U_9(params, wires=[i, i + 1])
        for i in range(1, 7, 2):
            U_9(params, wires=[i, i + 1])
    else:
        print("Invalid Unitary Ansatze")
        return False

#How to visualize from the architecture??
def conv_layer2(U, params):
    """U(params, wires=[0, 6]) # applies the unitary operation U to qubits 0 and 6
    U(params, wires=[0, 2]) #unitary operation U to qubits 0 and 2.
    U(params, wires=[4, 6]) #unitary operation is applied to qubits 4 and 6.
    U(params, wires=[2, 4])"""
    if U == 'U_TTN':
        U_TTN(params, wires=[0, 6])
        U_TTN(params, wires=[0, 2])
        U_TTN(params, wires=[4, 6])
        U_TTN(params, wires=[2, 4])
    elif U == 'U_5':
        U_5(params, wires=[0, 6])
        U_5(params, wires=[0, 2])
        U_5(params, wires=[4, 6])
        U_5(params, wires=[2, 4])
    elif U == 'U_6':
        U_6(params, wires=[0, 6])
        U_6(params, wires=[0, 2])
        U_6(params, wires=[4, 6])
        U_6(params, wires=[2, 4])
    elif U == 'U_9':
        U_9(params, wires=[0, 6])
        U_9(params, wires=[0, 2])
        U_9(params, wires=[4, 6])
        U_9(params, wires=[2, 4])
    elif U == 'U_13':
        U_13(params, wires=[0, 6])
        U_13(params, wires=[0, 2])
        U_13(params, wires=[4, 6])
        U_13(params, wires=[2, 4])
    elif U == 'U_14':
        U_14(params, wires=[0, 6])
        U_14(params, wires=[0, 2])
        U_14(params, wires=[4, 6])
        U_14(params, wires=[2, 4])
    elif U == 'U_15':
        U_15(params, wires=[0, 6])
        U_15(params, wires=[0, 2])
        U_15(params, wires=[4, 6])
        U_15(params, wires=[2, 4])
    elif U == 'U_SO4':
        U_SO4(params, wires=[0, 6])
        U_SO4(params, wires=[0, 2])
        U_SO4(params, wires=[4, 6])
        U_SO4(params, wires=[2, 4])
    elif U == 'U_SU4':
        U_SU4(params, wires=[0, 6])
        U_SU4(params, wires=[0, 2])
        U_SU4(params, wires=[4, 6])
        U_SU4(params, wires=[2, 4])
    elif U == 'U_SU4_no_pooling':
        U_SU4(params, wires=[0, 6])
        U_SU4(params, wires=[0, 2])
        U_SU4(params, wires=[4, 6])
        U_SU4(params, wires=[2, 4])
    elif U == 'U_SU4_1D':
        U_SU4(params, wires=[0, 6])
        U_SU4(params, wires=[0, 2])
        U_SU4(params, wires=[4, 6])
        U_SU4(params, wires=[2, 4])
    elif U == 'U_9_1D':
        U_9(params, wires=[0, 6])
        U_9(params, wires=[0, 2])
        U_9(params, wires=[4, 6])
        U_9(params, wires=[2, 4])
    else:
        print("Invalid Unitary Ansatze")
        return False

def conv_layer3(U, params):
    """U(params, wires=[0,4])"""
    if U == 'U_TTN':
        U_TTN(params, wires=[0, 4])
    elif U == 'U_5':
        U_5(params, wires=[0, 4])
    elif U == 'U_6':
        U_6(params, wires=[0, 4])
    elif U == 'U_9':
        U_9(params, wires=[0, 4])
    elif U == 'U_13':
        U_13(params, wires=[0, 4])
    elif U == 'U_14':
        U_14(params, wires=[0, 4])
    elif U == 'U_15':
        U_15(params, wires=[0, 4])
    elif U == 'U_SO4':
        U_SO4(params, wires=[0, 4])
    elif U == 'U_SU4':
        U_SU4(params, wires=[0, 4])
    elif U == 'U_SU4_no_pooling':
        U_SU4(params, wires=[0, 4])
    elif U == 'U_SU4_1D':
        U_SU4(params, wires=[0, 4])
    elif U == 'U_9_1D':
        U_9(params, wires=[0, 4])
    else:
        print("Invalid Unitary Ansatze3")
        return False

# Quantum Circuits for Pooling layers
def pooling_layer1(V, params):
    '''for i in range(0, 8, 2):
        V(params, wires=[i + 1, i]) #wire 1 and 0, wire 3 and 2, wire 5 and 4, wire 7 and 6'''

    if V == 'Pooling_ansatz1':
        for i in range(0, 8, 2):
            Pooling_ansatz1(params, wires=[i + 1, i])
    elif V == 'Pooling_ansatz2':
        for i in range(0, 8, 2):
            Pooling_ansatz2(wires=[i + 1, i]) #no params
    elif V == 'Pooling_ansatz3':
        for i in range(0, 8, 2):
            Pooling_ansatz3(*params, wires=[i + 1, i]) #not limited params
    elif V=='custom_Pooling4':
            custom_Pooling4(params,wires=[0,1,2,3])
            custom_Pooling4(params,wires=[4,5,6,7])
    elif V == 'custom_Pooling1':
        custom_Pooling1(params, wires=[0, 1, 2, 3])
        custom_Pooling1(params, wires=[4, 5, 6, 7])
    elif V == 'custom_Pooling3_Mod_a':
        custom_Pooling3_Mod_a(params, wires=[0, 1, 2, 3])
        custom_Pooling3_Mod_a(params, wires=[4, 5, 6, 7])
    elif V == 'custom_Pooling3_Mod_b':
        custom_Pooling3_Mod_b(params, wires=[0, 1, 2, 3])
        custom_Pooling3_Mod_b(params, wires=[4, 5, 6, 7])
    elif V == 'custom_ansatz3_Mod_c':
        custom_ansatz3_Mod_c(params, wires=[0, 1, 2, 3])
        custom_ansatz3_Mod_c(params, wires=[4, 5, 6, 7])
    else:
        print("Invalid Pooling Ansatze")
        return False

        #how to visualize??

"""def pooling_layer2(V, params):
    V(params, wires=[2,0])
    V(params, wires=[6,4])"""
def pooling_layer2(V, params):
    if V == 'Pooling_ansatz1':
        Pooling_ansatz1(params, wires=[2, 0])
        Pooling_ansatz1(params, wires=[6, 4])
    elif V == 'Pooling_ansatz2':
        Pooling_ansatz2(wires=[2, 0])
        Pooling_ansatz2(wires=[6, 4])
    elif V == 'Pooling_ansatz3':
        Pooling_ansatz3(*params, wires=[2, 0])
        Pooling_ansatz3(*params, wires=[6, 4])
    elif V=='custom_Pooling4':
        custom_Pooling4(params,wires=[0,2,4,6]) #using custom_pooling1
    elif V == 'custom_Pooling1':
        custom_Pooling1(params, wires=[0, 2, 4, 6])
    elif V == 'custom_Pooling3_Mod_a':
        custom_Pooling3_Mod_a(params, wires=[0, 2, 4, 6])
    elif V == 'custom_Pooling3_Mod_b':
        custom_Pooling3_Mod_b(params, wires=[0, 2, 4, 6])
    elif V == 'custom_ansatz3_Mod_c':
        custom_ansatz3_Mod_c(params, wires=[0, 2, 4, 6])
    else:
        print("Invalid Pooling Ansatze")
        return False

"""def pooling_layer3(V, params):
    V(params, wires=[0,4])"""

def pooling_layer3(V, params):
    if V == 'Pooling_ansatz1':
        Pooling_ansatz1(params, wires=[0, 4])
    elif V == 'Pooling_ansatz2':
        Pooling_ansatz2(wires=[0, 4])
    elif V == 'Pooling_ansatz3':
        Pooling_ansatz3(*params, wires=[0, 4])
    else:
        print("Invalid Pooling Ansatze")
        return False



def QCNN_structure(U, params, U_params):
    '''U: A unitary operation that will be applied in the convolutional layers.
    params: A list of parameters to be used in the unitary operations.
    U_params: The number of parameters expected for each unitary operation.'''
    param1 = params[0:U_params[0]]
    param2 = params[U_params[0]: U_params[0]+U_params[2]]
    param3 = params[U_params[0]+U_params[2]: U_params[0]+U_params[2]+U_params[4]]
    param4 = params[U_params[0]+U_params[2]+U_params[4]: U_params[0]+U_params[2]+U_params[4]+U_params[1]]
    param5 = params[U_params[0]+U_params[2]+U_params[4]+U_params[1]: U_params[0]+U_params[2]+U_params[4]+U_params[1]+U_params[3]]
    param6 = params[U_params[0]+U_params[2]+U_params[4]+U_params[1]+U_params[3]: U_params[0]+U_params[2]+U_params[4]+U_params[1]+U_params[3]+U_params[5]]

    # Pooling Ansatz1 is used by default
    #3 conv, 3 pooling
    conv_layer1(U[0], param1)
    pooling_layer1(U[1], param4)
    conv_layer2(U[2], param2)
    pooling_layer2(U[3], param5)
    conv_layer3(U[4], param3)
    pooling_layer3(U[5], param6)


def QCNN_structure_without_pooling(U, params, U_params):
    param1 = params[0:U_params[0]]
    param2 = params[U_params[0]: U_params[0]+U_params[2]]
    param3 = params[U_params[0]+U_params[2]: U_params[0]+U_params[2]+U_params[4]]
    #No_Pooling used
    #3 conv
    conv_layer1(U[0], param1)
    conv_layer2(U[2], param2)
    conv_layer3(U[4], param3)

def QCNN_1D_circuit(U, params, U_params):#Assuming all U[0],U[2] and U[4] has QCNN_1D_circuit
    """param1 = params[0: U_params]
    param2 = params[U_params: 2*U_params]
    param3 = params[2*U_params: 3*U_params]"""
    param1 = params[0:U_params[0]]
    param2 = params[U_params[0]: U_params[0]+U_params[2]]
    param3 = params[U_params[0]+U_params[2]: U_params[0]+U_params[2]+U_params[4]]

    if U[0]=='U_SU4_1D': #just check for 1 value
        for i in range(0, 8, 2):
            U_SU4(param1, wires=[i, i + 1]) #wire 0 and 1, wire 2 and 3, wire 4 and 5, wire 6 and 7
        for i in range(1, 7, 2):
            U_SU4(param1, wires=[i, i + 1]) #wire 1 and 2, wire 3 and 4, wire 5 and 6

        U_SU4(param2, wires=[2,3]) #unitary operation U to qubits 2 and 3. Visuaalize??
        U_SU4(param2, wires=[4,5])
        U_SU4(param3, wires=[3,4])
        
    elif U[0]=='U_9_1D':
        for i in range(0, 8, 2):
            U_9(param1, wires=[i, i + 1]) #wire 0 and 1, wire 2 and 3, wire 4 and 5, wire 6 and 7
        for i in range(1, 7, 2):
            U_9(param1, wires=[i, i + 1]) #wire 1 and 2, wire 3 and 4, wire 5 and 6

        U_9(param2, wires=[2,3]) #unitary operation U to qubits 2 and 3. Visuaalize??
        U_9(param2, wires=[4,5])
        U_9(param3, wires=[3,4])

    



dev = qml.device('default.qubit', wires = 8) # A quantum device with 8 qubits is initialized using PennyLane's default qubit simulator.
@qml.qnode(dev) #The QCNN function is decorated as a QNode. This means it will execute on the specified quantum device (dev) and can compute quantum states or observables based on classical inputs.


def QCNN(X, params, U, U_params, embedding_type='Amplitude', cost_fn='cross_entropy'):


    # Data Embedding
    data_embedding(X, embedding_type=embedding_type)

    # Quantum Convolutional Neural Network
    """if U == 'U_TTN':
        QCNN_structure(U_TTN, params, U_params)
    elif U == 'U_5':
        QCNN_structure(U_5, params, U_params)
    elif U == 'U_6':
        QCNN_structure(U_6, params, U_params)
    elif U == 'U_9':
        QCNN_structure(U_9, params, U_params)
    elif U == 'U_13':
        QCNN_structure(U_13, params, U_params)
    elif U == 'U_14':
        QCNN_structure(U_14, params, U_params)
    elif U == 'U_15':
        QCNN_structure(U_15, params, U_params)
    elif U == 'U_SO4':
        QCNN_structure(U_SO4, params, U_params)
    elif U == 'U_SU4':
        QCNN_structure(U_SU4, params, U_params)
    elif U == 'U_SU4_no_pooling':
        QCNN_structure_without_pooling(U_SU4, params, U_params)
    elif U == 'U_SU4_1D':
        QCNN_1D_circuit(U_SU4, params, U_params)
    elif U == 'U_9_1D':
        QCNN_1D_circuit(U_9, params, U_params)
    else:
        print("Invalid Unitary Ansatze")
        return False"""
    if 'U_SU4_no_pooling' in [U[0], U[2], U[4]]: #check if we have U_SU4_no_pooling in all3 layers
        QCNN_structure_without_pooling(U, params, U_params)
    elif 'U_SU4_1D' in [U[0], U[2], U[4]]:
        QCNN_1D_circuit(U, params, U_params)
    elif 'U_9_1D' in [U[0], U[2], U[4]]:
        QCNN_1D_circuit(U, params, U_params)
    elif ('U_SU4_no_pooling' not in U) and ('U_SU4_1D' not in U) and ('U_9_1D' not in U):
        QCNN_structure(U,params,U_params)
    else:
        print("Invalid Unitary Ansatze")


    
    if cost_fn == 'mse':
        result = qml.expval(qml.PauliZ(4))
    elif cost_fn == 'cross_entropy':
        result = qml.probs(wires=4)
    return result


# Hierarchial Circuit

---

In [47]:
# Implementaion of Hierarchical Quantum Classifier Structure.
import pennylane as qml
# import unitary
# import embedding

dev_TTN = qml.device('default.qubit', wires=8)

def Hierarchical_structure(U, params, U_params):
    param1 = params[0 * U_params:1 * U_params]
    param2 = params[1 * U_params:2 * U_params]
    param3 = params[2 * U_params:3 * U_params]
    param4 = params[3 * U_params:4 * U_params]
    param5 = params[4 * U_params:5 * U_params]
    param6 = params[5 * U_params:6 * U_params]
    param7 = params[6 * U_params:7 * U_params]

    # 1st Layer
    U(param1, wires=[0, 1])
    U(param2, wires=[2, 3])
    U(param3, wires=[4, 5])
    U(param4, wires=[6, 7])
    # 2nd Layer
    U(param5, wires=[1, 3])
    U(param6, wires=[5, 7])
    # 3rd Layer
    U(param7, wires=[3, 7])



@qml.qnode(dev_TTN)
def Hierarchical_classifier(X, params, U, U_params, embedding_type='Amplitude', cost_fn='cross_entropy'):
    data_embedding(X, embedding_type=embedding_type)
    if U == 'U_TTN':
        Hierarchical_structure(U_TTN, params, U_params)
    elif U == 'U_5':
        Hierarchical_structure(U_5, params, U_params)
    elif U == 'U_6':
        Hierarchical_structure(U_6, params, U_params)
    elif U == 'U_9':
        Hierarchical_structure(U_9, params, U_params)
    elif U == 'U_13':
        Hierarchical_structure(U_13, params, U_params)
    elif U == 'U_14':
        Hierarchical_structure(U_14, params, U_params)
    elif U == 'U_15':
        Hierarchical_structure(U_15, params, U_params)
    elif U == 'U_SO4':
        Hierarchical_structure(U_SO4, params, U_params)
    elif U == 'U_SU4':
        Hierarchical_structure(U_SU4, params, U_params)
    else:
        print("Invalid Unitary Ansatz")
        return False
    if cost_fn == 'mse':
        result = qml.expval(qml.PauliZ(7))
    elif cost_fn == 'cross_entropy':
        result = qml.probs(wires=7)
    return result


# Training

---

In [48]:
# Implementation of Quantum circuit training procedure
# import QCNN_circuit
# import Hierarchical_circuit
import pennylane as qml
from pennylane import numpy as np
import autograd.numpy as anp
import pennylane.numpy as qnp #new line added 11/25/24


def square_loss(labels, predictions):
    #It iterates through each pair of true label l and predicted value p, calculating the squared difference.
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2
    #The total loss is averaged over the number of labels.
    loss = loss / len(labels)
    return loss

def cross_entropy(labels, predictions):
    #Computes the cross-entropy loss, commonly used in binary classification tasks.
    loss = 0
    for l, p in zip(labels, predictions): #For each label-prediction pair, it calculates the cross-entropy based on whether the label is 0 or 1.
        c_entropy = l * (anp.log(p[l])) + (1 - l) * anp.log(1 - p[1 - l])
        loss = loss + c_entropy
    return -1 * loss #The total loss is returned as a negative value because cross-entropy is typically minimized.






def cost(params, X, Y, U, U_params, embedding_type, circuit, cost_fn):
    if circuit == 'QCNN':
        predictions = [QCNN(x, params, U, U_params, embedding_type, cost_fn=cost_fn) for x in X]
    elif circuit == 'Hierarchical':
        predictions = [Hierarchical_classifier(x, params, U, U_params, embedding_type, cost_fn=cost_fn) for x in X]

    if cost_fn == 'mse':
        loss = square_loss(Y, predictions)
    elif cost_fn == 'cross_entropy':
        loss = cross_entropy(Y, predictions)
    return loss




# Circuit training parameters
'''
steps: Number of iterations for training.
learning_rate: Step size for parameter updates during optimization.
batch_size: Number of samples used in each iteration of training.'''
steps = 200
learning_rate = 0.01
batch_size = 25


def circuit_training(X_train, Y_train, U, U_params, embedding_type, circuit, cost_fn):
    if circuit == 'QCNN':
        total_params=0
        for i in U_params:
            if i!="":
                total_params+=int(i)
        #print(total_params)


    elif circuit == 'Hierarchical':
        #total_params = U_params * 7
        pass
    # params = qnp.random.randn(total_params, requires_grad=True)
    params = qnp.random.randn(total_params, requires_grad=True)
    opt = qml.NesterovMomentumOptimizer(stepsize=learning_rate)
    loss_history = []

    for it in range(steps):
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [Y_train[i] for i in batch_index]
        params, cost_new = opt.step_and_cost(lambda v: cost(v, X_batch, Y_batch, U, U_params, embedding_type, circuit, cost_fn),
                                                     params)
        loss_history.append(cost_new)
        if it % 10 == 0:
            print("iteration: ", it, " cost: ", cost_new)
    return loss_history, params




# Benchmarking
---

In [49]:
# import data
# import Training
# import QCNN_circuit
# import Hierarchical_circuit
import numpy as np

def accuracy_test(predictions, labels, cost_fn, binary = True):
        #predictions: The predicted values from the model. labels: The true labels for the data. cost_fn: A string indicating the type of cost function used in training ('mse' for mean squared error or            'cross_entropy' for cross-entropy). binary: A boolean flag indicating whether the classification is binary (default is True).
    if cost_fn == 'mse':
        '''Binary Classification: If binary is True, it initializes an accuracy counter (acc) to zero.
        It iterates through each pair of true labels (l) and predicted values (p).
        It checks if the absolute difference between the true label and prediction is less than 1. This means that as long as the prediction is within 1 unit of the true label (which generally should be either 0 or 1), it counts as a correct prediction.
         The accuracy is then calculated as the number of correct predictions divided by the total number of labels.'''
        if binary == True:
            acc = 0
            for l, p in zip(labels, predictions):
                if np.abs(l - p) < 1:
                    acc = acc + 1
            return acc / len(labels)

        else:
            """Non-Binary Classification: If binary is False, it again initializes an accuracy counter.It checks if the absolute difference between the true label and prediction is less than 0.5. This allows for a wider margin of error in non-binary classifications (e.g., multi-class scenarios)."""
            acc = 0
            for l, p in zip(labels, predictions):
                if np.abs(l - p) < 0.5:
                    acc = acc + 1
            return acc / len(labels)

    elif cost_fn == 'cross_entropy':
        acc = 0
        for l,p in zip(labels, predictions):
            if p[0] > p[1]:
                P = 0
            else:
                P = 1
            if P == l:
                acc = acc + 1
        return acc / len(labels)


def Encoding_to_Embedding(Encoding):
    # Amplitude Embedding / Angle Embedding
    if Encoding == 'resize256':
        Embedding = 'Amplitude'
    elif Encoding == 'pca8':
        Embedding = 'Angle'
    elif Encoding == 'autoencoder8':
        Embedding = 'Angle'

    # Amplitude Hybrid Embedding
    # 4 qubit block
    elif Encoding == 'pca80-1':
        Embedding = 'Amplitude-Hybrid4-1'
    elif Encoding == 'autoencoder80-1':
        Embedding = 'Amplitude-Hybrid4-1'

    elif Encoding == 'pca80-2':
        Embedding = 'Amplitude-Hybrid4-2'
    elif Encoding == 'autoencoder80-2':
        Embedding = 'Amplitude-Hybrid4-2'

    elif Encoding == 'pca80-3':
        Embedding = 'Amplitude-Hybrid4-3'
    elif Encoding == 'autoencoder80-3':
        Embedding = 'Amplitude-Hybrid4-3'

    elif Encoding == 'pca80-4':
        Embedding = 'Amplitude-Hybrid4-4'
    elif Encoding == 'autoencoder80-4':
        Embedding = 'Amplitude-Hybrid4-4'





    elif Encoding =='pca60-1':
        Embedding = 'Amplitude-Hybrid4-1'
    elif Encoding == 'autoencoder60-1':
        Embedding = 'Amplitude-Hybrid4-1'
    
    elif Encoding =='pca60-2':
        Embedding = 'Amplitude-Hybrid4-2'
    elif Encoding == 'autoencoder60-2':
        Embedding = 'Amplitude-Hybrid4-2'
    
    elif Encoding =='pca60-3':
        Embedding = 'Amplitude-Hybrid4-3'
    elif Encoding == 'autoencoder60-3':
        Embedding = 'Amplitude-Hybrid4-3'
    elif Encoding =='pca60-4':
        Embedding = 'Amplitude-Hybrid4-4'
    elif Encoding == 'autoencoder60-4':
        Embedding = 'Amplitude-Hybrid4-4'



    elif Encoding == 'pca32-1':
        Embedding = 'Amplitude-Hybrid4-1'
    elif Encoding == 'autoencoder32-1':
        Embedding = 'Amplitude-Hybrid4-1'

    elif Encoding == 'pca32-2':
        Embedding = 'Amplitude-Hybrid4-2'
    elif Encoding == 'autoencoder32-2':
        Embedding = 'Amplitude-Hybrid4-2'

    elif Encoding == 'pca32-3':
        Embedding = 'Amplitude-Hybrid4-3'
    elif Encoding == 'autoencoder32-3':
        Embedding = 'Amplitude-Hybrid4-3'

    elif Encoding == 'pca32-4':
        Embedding = 'Amplitude-Hybrid4-4'
    elif Encoding == 'autoencoder32-4':
        Embedding = 'Amplitude-Hybrid4-4'

    # 2 qubit block
    elif Encoding == 'pca16-1':
        Embedding = 'Amplitude-Hybrid2-1'
    elif Encoding == 'autoencoder16-1':
        Embedding = 'Amplitude-Hybrid2-1'

    elif Encoding == 'pca16-2':
        Embedding = 'Amplitude-Hybrid2-2'
    elif Encoding == 'autoencoder16-2':
        Embedding = 'Amplitude-Hybrid2-2'

    elif Encoding == 'pca16-3':
        Embedding = 'Amplitude-Hybrid2-3'
    elif Encoding == 'autoencoder16-3':
        Embedding = 'Amplitude-Hybrid2-3'

    elif Encoding == 'pca16-4':
        Embedding = 'Amplitude-Hybrid2-4'
    elif Encoding == 'autoencoder16-4':
        Embedding = 'Amplitude-Hybrid2-4'

    # Angular HybridEmbedding
    # 4 qubit block
    elif Encoding == 'pca30-1':
        Embedding = 'Angular-Hybrid4-1'
    elif Encoding == 'autoencoder30-1':
        Embedding = 'Angular-Hybrid4-1'

    elif Encoding == 'pca30-2':
        Embedding = 'Angular-Hybrid4-2'
    elif Encoding == 'autoencoder30-2':
        Embedding = 'Angular-Hybrid4-2'

    elif Encoding == 'pca30-3':
        Embedding = 'Angular-Hybrid4-3'
    elif Encoding == 'autoencoder30-3':
        Embedding = 'Angular-Hybrid4-3'

    elif Encoding == 'pca30-4':
        Embedding = 'Angular-Hybrid4-4'
    elif Encoding == 'autoencoder30-4':
        Embedding = 'Angular-Hybrid4-4'

    # 2 qubit block
    elif Encoding == 'pca12-1':
        Embedding = 'Angular-Hybrid2-1'
    elif Encoding == 'autoencoder12-1':
        Embedding = 'Angular-Hybrid2-1'

    elif Encoding == 'pca12-2':
        Embedding = 'Angular-Hybrid2-2'
    elif Encoding == 'autoencoder12-2':
        Embedding = 'Angular-Hybrid2-2'

    elif Encoding == 'pca12-3':
        Embedding = 'Angular-Hybrid2-3'
    elif Encoding == 'autoencoder12-3':
        Embedding = 'Angular-Hybrid2-3'

    elif Encoding == 'pca12-4':
        Embedding = 'Angular-Hybrid2-4'
    elif Encoding == 'autoencoder12-4':
        Embedding = 'Angular-Hybrid2-4'

    # Two Gates Compact Encoding
    elif Encoding == 'pca16-compact':
        Embedding = 'Angle-compact'
    elif Encoding == 'autoencoder16-compact':
        Embedding = 'Angle-compact'
    return Embedding


def Benchmarking(dataset, classes, Unitaries, U_num_params, Encodings, circuit, cost_fn, binary=True):
    '''I: The number of different unitary operations.
        J: The number of different encoding methods.'''
    I = len(Unitaries)
    J = len(Encodings)


    '''The outer loop iterates over each unitary operation (U), and the inner loop iterates over each encoding method (Encoding).'''
    for i in range(I):
        for j in range(J):
            f = open('result.txt', 'a+') #A file named result.txt is opened in append mode ('a+') to log results.
            U = Unitaries[i]
            U_params = U_num_params[i]
            Encoding = Encodings[j]
            Embedding = Encoding_to_Embedding(Encoding)

            X_train, X_test, Y_train, Y_test = data_load_and_process(dataset, classes=classes,
                                                                          feature_reduction=Encoding, binary=binary)

            print("\n")
            print("Loss History for " + circuit + " circuits, " + str(U) + " " + Encoding + " with " + cost_fn)
            loss_history, trained_params = circuit_training(X_train, Y_train, U, U_params, Embedding, circuit, cost_fn)

            #QCNN
            if circuit == 'QCNN':
                predictions = [QCNN(x, trained_params, U, U_params, Embedding, cost_fn) for x in X_test]


            elif circuit == 'Hierarchical':
                predictions = [Hierarchical_classifier(x, trained_params, U, U_params, Embedding, cost_fn) for x in X_test]

            #Accuracy
            accuracy = accuracy_test(predictions, Y_test, cost_fn, binary)
            print("Accuracy for " + str(U) + " " + Encoding + " :" + str(accuracy))

            f.write("Loss History for " + circuit + " circuits, " + str(U) + " " + Encoding + " with " + cost_fn)
            f.write("\n")
            f.write(str(loss_history))
            f.write("\n")
            f.write("Accuracy for " + str(U) + " " + Encoding + " :" + str(accuracy))
            f.write("\n")
            f.write("\n")
    f.close()





def Data_norm(dataset, classes, Encodings, binary=True):
    '''dataset: The dataset to be processed.
    classes: The classes or labels associated with the dataset.
    Encodings: A list of encoding methods to be used (e.g., PCA, autoencoders).
    binary: A boolean indicating whether the classification task is binary (default is True).'''
    J = len(Encodings) #J: The number of different encoding methods provided in the Encodings list.
    Num_data = 10000 #Num_data: A constant set to 10,000, representing the number of samples to be randomly selected for normalization calculations.

    f = open('Result/data_norm.txt', '+a')

    for j in range(J):
        Encoding = Encodings[j]

        X_train, X_test, Y_train, Y_test = data_load_and_process(dataset, classes=classes,
                                                                          feature_reduction=Encoding, binary=binary)

        #norm calculation for pca32-3 and autoencoder32-3
        if Encoding == 'pca32-3' or Encoding == 'autoencoder32-3':

            '''-> Norm Calculation:
                Initializes two lists (norms_X1, norms_X2) to store the norms for two segments of each selected sample.
                For each iteration up to Num_data, it randomly selects an index from X_train, retrieves the corresponding sample (X), and splits it into two parts (X1 and X2).
                It calculates the L2 norm (Euclidean norm) for both segments using np.linalg.norm() and appends these values to their respective lists.
               -> Statistics Calculation:
                After collecting norms for all samples, it converts the lists into NumPy arrays and computes their means and standard deviations.
                -> Logging Results:
                Writes the encoding type and calculated statistics (mean and standard deviation) into the results file.'''
            norms_X1 = []
            norms_X2 = []
            for i in range(Num_data):
                index = np.random.randint(0, len(X_train))
                X = X_train[index]

                X1 = X[:2 ** 4]
                X2 = X[2 ** 4:2 ** 5]
                norm_X1, norm_X2 = np.linalg.norm(X1), np.linalg.norm(X2)
                norms_X1.append(norm_X1)
                norms_X2.append(norm_X2)

            norms_X1, norms_X2 = np.array(norms_X1), np.array(norms_X2)
            mean_X1, stdev_X1 = np.mean(norms_X1), np.std(norms_X1)
            mean_X2, stdev_X2 = np.mean(norms_X2), np.std(norms_X2)

            if Encoding == 'pca32-3':
                f.write("PCA32 Encoding\n")
            elif Encoding == 'autoencoder32-3':
                f.write("autoencoder32 Encoding\n")
            f.write("mean of X1: " + str(mean_X1) + " standard deviation of X1: " + str(stdev_X1))
            f.write("\n")
            f.write("mean of X2: " + str(mean_X2) + " standard deviation of X2: " + str(stdev_X2))
            f.write("\n")

        #if pca16 or autoencoder16 is used

        elif Encoding == 'pca16' or Encoding == 'autoencoder16':

            '''This section follows a similar logic as above but handles four segments instead of two.

                It initializes four lists for storing norms (norms_X1, norms_Y, etc.) corresponding to each segment.

                The process involves random sampling from X_train, calculating L2 norms for each segment, computing means and standard deviations for all segments, and logging results.'''
            norms_X1 = []
            norms_X2 = []
            norms_X3 = []
            norms_X4 = []
            for i in range(Num_data):
                index = np.random.randint(0, len(X_train))
                X = X_train[index]

                X1 = X[:4]
                X2 = X[4:8]
                X3 = X[8:12]
                X4 = X[12:16]
                norm_X1, norm_X2, norm_X3, norm_X4 = np.linalg.norm(X1), np.linalg.norm(X2), np.linalg.norm(
                    X3), np.linalg.norm(X4)

                norms_X1.append(norm_X1)
                norms_X2.append(norm_X2)
                norms_X3.append(norm_X3)
                norms_X4.append(norm_X4)

            norms_X1, norms_X2, norms_X3, norms_X4 = np.array(norms_X1), np.array(norms_X2), np.array(norms_X3), np.array(norms_X4)

            mean_X1, stdev_X1 = np.mean(norms_X1), np.std(norms_X1)
            mean_X2, stdev_X2 = np.mean(norms_X2), np.std(norms_X2)
            mean_X3, stdev_X3 = np.mean(norms_X3), np.std(norms_X3)
            mean_X4, stdev_X4 = np.mean(norms_X4), np.std(norms_X4)

            if Encoding == 'pca16':
                f.write("PCA16 Encoding\n")
            elif Encoding == 'autoencoder16':
                f.write("autoencoder16 Encoding\n")
            f.write("mean of X1: " + str(mean_X1) + " standard deviation of X1: " + str(stdev_X1))
            f.write("\n")
            f.write("mean of X2: " + str(mean_X2) + " standard deviation of X2: " + str(stdev_X2))
            f.write("\n")
            f.write("mean of X3: " + str(mean_X3) + " standard deviation of X3: " + str(stdev_X3))
            f.write("\n")
            f.write("mean of X4: " + str(mean_X4) + " standard deviation of X4: " + str(stdev_X4))
            f.write("\n")

    f.close()


# Result

---

In [50]:
# This generates the results of the bechmarking code

# import Benchmarking


"""
Here are possible combinations of benchmarking user could try.
Unitaries: ['U_TTN', 'U_5', 'U_6', 'U_9', 'U_13', 'U_14', 'U_15', 'U_SO4', 'U_SU4', 'U_SU4_no_pooling', 'U_SU4_1D', 'U_9_1D']
U_num_params: [2, 10, 10, 2, 6, 6, 4, 6, 15, 15, 15, 2]
Encodings: ['resize256', 'pca8', 'autoencoder8', 'pca16-compact', 'autoencoder16-compact', 'pca32-1', 'autoencoder32-1',
            'pca16-1', 'autoencoder16-1', 'pca30-1', 'autoencoder30-1', 'pca12-1', 'autoencoder12-1']
dataset: 'mnist' or 'fashion_mnist'
circuit: 'QCNN' or 'Hierarchical'
cost_fn: 'mse' or 'cross_entropy'
Note: when using 'mse' as cost_fn binary="True" is recommended, when using 'cross_entropy' as cost_fn must be binary="False".
"""

"""Conv_Pooling_Layer_Ansatzs = [['U_SU4', 'Pooling_ansatz1','U_SU4', 'Pooling_ansatz1', 'U_SU4','Pooling_ansatz1'], ['U_5','Pooling_ansatz1','U_5','Pooling_ansatz1','U_5','Pooling_ansatz1'],['U_SU4_1D',"",'U_SU4_1D',"",'U_SU4_1D',""]]
Conv_Pooling_Layer_Params = [[15, 2, 15, 2, 15, 2],[10,2,10,2,10,2],[15,"",15,"",15]]"""



#Format
#Conv_Pooling_Layer_Ansatzs = [[Convolution Layer 1 Ansatz,Pooling Layer1 Ansatz, Convolution  Layer 2 Ansatz, Pooling Layer 2 Ansatz, Convolution Layer 3 Ansatz, Pooling Layer 3 Ansatz], [2nd Loop],[3rd Loop],........]
#Conv_Pooling_Layer_Params =[[Convolution Layer 1 parameter,Pooling Layer1 parameter, Convolution  Layer 2 parameter, Pooling Layer 2 parameter, Convolution Layer 3 parameter, Pooling Layer 3 parameter]]

Conv_Pooling_Layer_Ansatzs = [['U_5',"custom_Pooling1",'U_5','custom_Pooling4','U_5','Pooling_ansatz1']]
Conv_Pooling_Layer_Params = [[10,10,10,14,10,2]]
Encodings = ['autoencoder80-4','pca80-4'] 
dataset = 'Dataset\Mel_Spectrograms'
classes = [0,1]
binary = False
cost_fn = 'cross_entropy'

Benchmarking(dataset, classes,Conv_Pooling_Layer_Ansatzs, Conv_Pooling_Layer_Params, Encodings, circuit='QCNN', cost_fn=cost_fn, binary=binary)
#Benchmarking.Benchmarking(dataset, classes, Unitaries, U_num_params, Encodings, circuit='Hierarchical', cost_fn=cost_fn, binary=binary)



X_train shape: (80, 370, 969, 1), Y_train shape: (80,)
X_test shape: (20, 370, 969, 1), Y_test shape: (20,)
Normalized
Loaded X_train shape: (80, 370, 969, 1), Y_train shape: (80,)
Loaded X_test shape: (20, 370, 969, 1), Y_test shape: (20,)
Epoch 1/10
3/3 [==============================] - 2s 554ms/step - loss: 0.0429 - val_loss: 0.0392
Epoch 2/10
3/3 [==============================] - 1s 494ms/step - loss: 0.0425 - val_loss: 0.0368
Epoch 3/10
3/3 [==============================] - 1s 464ms/step - loss: 0.0370 - val_loss: 0.0288
Epoch 4/10
3/3 [==============================] - 1s 465ms/step - loss: 0.0337 - val_loss: 0.0254
Epoch 5/10
3/3 [==============================] - 1s 454ms/step - loss: 0.0297 - val_loss: 0.0276
Epoch 6/10
3/3 [==============================] - 1s 461ms/step - loss: 0.0343 - val_loss: 0.0227
Epoch 7/10
3/3 [==============================] - 1s 474ms/step - loss: 0.0267 - val_loss: 0.0234
Epoch 8/10
3/3 [==============================] - 1s 448ms/step - loss: 0